# Assignment  week 7

## Notebook made by   (If not filled in correctly: 0 pts for assignment)

__Name(s)__: Nora Schinkel & Maarten Boon

__Student id(s)__ : 10416730 & 10764399

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. **Assignments without the selfies will not be graded and receive 0 points.**

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='http://i.imgur.com/9g88in2.jpg%5C'/>
<img src='https://raw.githubusercontent.com/noracato/Kennissystemen/master/pledge.jpg'/>

# Infoboxes

* We willen voor elke Wikipedia pagina de informatie uit de infobox halen en die gestructureerd opslaan.
* In het simpele geval is de infobox een lijst van attribuut-waarde paren:
    * het zijn attributen van de "entiteit" waar de pagina over gaat.
    * Dit zijn dus typisch _RDF subject-predicate-object triples_
    * Die je in eerste orde logica als een binaire relatie zou opslaan (bijvoorbeeld _DateOfBirth(Albert-Speer,'19 maart 1905')_)
* Je zal zien dat hier flink wat haken en ogen aan zitten.
    



## Inspecteer een voorbeeld

* We gaan eens kijken hoe we de attribuut-waarde paren uit de zogenaamde _infoboxen_ uit de Wikipedia paginas kunnen gaan halen.
* Vergelijk eens het begin van de file `NLWikipedia/small.xml` met de pagina <https://nl.wikipedia.org/wiki/Albert_Speer>
* Die infobox informatie ziet er goed en redelijk eenvoudig te extraheren uit.
* Er zijn verschillende _mediawiki_ (het formaat van wikipedia paginas) parsers in omloop. 

### Je bent niet de eerste met deze opdracht ....

* Zoek op het web.
* Bijvoorbeeld: <http://stackoverflow.com/questions/8088226/content-of-infobox-of-wikipedia/21107068#21107068>
* Zie ook <https://www.mediawiki.org/wiki/Alternative_parsers> voor nog meer mediawiki parsers
* Denk ook eens aan <http://pandoc.org/demos.html>
* Zie ook [WikipediaInfoBoxes](WikipediaInfoBoxes) is de assignments folder.

## Vragen

## Download, en extraheer.  (33%)
* Download de dump met alle NL wikipedia paginas.


1. Verzin een goede datastructuur voor de subject-predicate-object triples die je uit de infoboxes per pagina gaat extraheren.
1. Schrijf een functie die uit het text element het infobox deel extraheert, en die lijst attribuut-waarde paren omzet naar jouw datastructuur.
2. Pas je functie toe op het [voorbeeld](NLWikipedia/sample.xml, en debug je code.

### Nou op de hele file/dump
1. Zoek uit hoe je een file stromend kunt inlezen in Python.
2. Pas nu je code toe op de hele file. 
3. Misschien  moet je wel  steeds gegevens naar een file gaan wegschrijven. 
4. Is jouw gekozen data formaat  handig?

#### Hints:
* Wil niet meteen alles tegelijk. Als er te veel variatie in de data zit, richt je dan op de meest voorkomende gevallen, en probeer die goed te krijgen. 
* Probeer wel een beeld te krijgen van "wat je mist", en beschrijf dat ook. 

## Analyseer  (33%)

1. Maak statistieken, wat zijn de meest gebruikte attributen, hoe vaak worden ze gebruikt, hoeveel attributen zijn er? Welke waardes worden er veel gebruikt? Hoeveel attributen zie je gemiddeld/mediaan per pagina? 
2. Doe een analyse op de waardes van de attributen? Kan je bepaalde types vinden?
3. Doe een analyse op de uniformiteit van de attribuutnamen. Uit de literatuur blijkt dat er erg veel verschillende manieren worden gebruikt om hetzelfde attribuut weer te geven (bijv, sterfdag, sterfdatum, sterftejaar, etc). Is dat ook zo in jouw dataset? Hoe kom je daar eigenlijk achter? 
 
 
## Reflecteer  (33%)
* Je hebt nu flink wat tijd besteed aan wat de mensen achter DBpedia, Yago, FreeBase en meer van dit soort initiatieven ook al doen. De bedoeling van de opdracht was om te ervaren wat een klus het is om een kennisbank te vullen met "gestructureerde" informatie van het web. 
* Beschrijf puntsgewijs wat volgens jullie de lastige onderdelen zijn van dit proces.
* Licht er 2-3 uit, en vertel hoe jullie het probleem hebben aangepakt.

# Mijn antwoorden

In [1]:
import requests
import json
import re
from bs4 import BeautifulSoup


class RDF(object):
    def __init__(self, subject, predicate, obj):
        self.s = subject
        self.p = predicate
        self.o = obj

def get_name(infobox):
    name = infobox[1]
    name = re.split('=', name)[1]
    return name

url = 'http://maartenmarx.nl/teaching/KennisSystemen/Assignments/NLWikipedia/small.xml'
r = requests.get(url) 
content = r.content
content = content.decode('UTF-8')
soup = BeautifulSoup(content, 'xml')
text = soup.get_text()
better_text = re.split(r'\n', text)
better_text = re.split('{{', text)

all_triples = []
for paragraphs in better_text:
    if paragraphs.startswith('Info'):
        infobox = re.split('}}', paragraphs)[0]
        infobox = re.split(r'\n', infobox)
        subject = get_name(infobox)
        infobox.pop(0)
        infobox.pop(0)
        goodinfo = []
        for info in infobox:
            if not info.endswith('=') and not info.endswith('= ') and not info == '' and info.startswith('|'):
                twoparts = re.split('=', info)
                predicate = twoparts[0]
                predicate = re.split(' ', predicate)[1]
                obj = twoparts[1]
                triple = RDF(subject, predicate, obj)
                all_triples.append(triple)
                
   
        


In [150]:
import matplotlib.pyplot as plt

#analyse all rdf type elem in a list to see most used attributes, most used values & how many different attributes
def analyse (rdflist):
    # puts all attributes and values from all objects in separate lists
    all_attributes = []
    all_values = []
    for rdf in rdflist:
        all_attributes.append(rdf.p)
        all_values.append(rdf.o)
    # make the calculations
    diff_atr = len(list(set(all_attributes)))
    plot1 = plot(all_attributes)
    plot2 = plot(all_values)
    print('Most used attribute is', plot1 , "times and the amount of different attributes is", diff_atr, ", the most used value is", plot2)

#make a plot of the 'voorkomens' of something in a list and return the item with the most 'voorkomens'
def plot(atlist):
    # get x and y for plot
    x = []
    y = []
    highest = [None, 0]
    
    # iterate through list and count the appearance of elements
    for atr in atlist:
        # if elem is in x, add 1 to appearence counter (y)
        if atr in x:
            y[x.index(atr)] +=1
        # if elem is not in x, add it and set counter y to 1
        else:
            x.append(atr)
            y.append(1)
     
    # deletes all things that only appear once from the plotlists
    for times in y:
        index = y.index(times)
        if times == 1:
            del y[index]
            del x[index]
        # checks for the highest appereance
        if times > highest[1]:
            highest[0] = x[index]
            highest[1] = times
    return highest
            
    # Start creating bar diagram        
    plt.bar(range(len(y)), y, color='b', label='Attributes')
    plt.xlabel(x)
    plt.ylabel('Times used')
    plt.legend()
    plt.show()

#run the analyse function, start analyzing!!
analyse(all_triples)

Most used attribute is ['afbeelding', 13] times and the amount of different attributes is 180 , the most used value is [' [[Europees Parlement]]', 2]


In [18]:
#Get the average of "used" attributes per page
def average_attribute(rdflist):
    all_attributes = []
    subjects = []
    for rdf in rdflist:
        all_attributes.append(rdf.p)
        subjects.append(rdf.s)
    total_attributes = len(all_attributes) #List of all attributes
    all_subjects = list(set(subjects)) #list of all different pages
    total_entries = len(all_subjects) 
    avg = total_attributes/total_entries
    return avg

print("average attributes per page", average_attribute(all_triples))

average attributes per page 12.409090909090908


In [141]:
#classifies all found values in the rdf triples
def classify_val(rdflist):
    # get all values to iterate over
    all_values = []
    for rdf in rdflist:
        all_values.append(rdf.o)
    # instantiate all different classification categories as lists
    all_dates = []
    images = []
    numbers = []
    links = []
    descriptions = []
    rest = []
    for val in all_values:
        # split value in parts for analyzing and remove any empty elements
        parts = re.split(' ', val)
        if '' in parts:
            parts.remove('')
        for part in parts:
            index = parts.index(part)
            part = part.strip('[')
            parts[index] = part.strip(']')
            
        # restoring to clean val
        cl_val = ' '.join(parts)
        
        length = len(parts)
        
        #check if date
        if length == 3 and RepresentsInt(parts[0]) and RepresentsInt(parts[2]):
            all_dates.append(cl_val)
        
        #check if time period
        elif length == 1 and '-' in parts[0] and RepresentsInt(parts[0][-1:]):
            all_dates.append(cl_val)
            
        #check if image
        elif val.endswith('.jpg') or val.endswith('.JPG')or val.endswith('.png'):
            images.append(cl_val)
        
        #check if number
        elif length == 1 and RepresentsInt(parts[0]): numbers.append(cl_val)
        
        #check if hyperlink (+description)
        elif parts[0].startswith('htt'): links.append(cl_val)
        
        #check if possibe description, criterium is to be longer than 3 words
        elif length > 3: descriptions.append(cl_val)
            
        else: rest.append(cl_val)
    
    
    print("Total found: dates ", len(all_dates), ", images ", len(images), ", lone numbers ", len(numbers), ", hyperlinks ", len(links)
         , ", descriptions ", len(descriptions), " and unclassified ", len(rest), "\n \n")
    
    print("All dates are :", all_dates, "\n")
    print("All images are:", images, "\n")
    print("All lone numbers are:", numbers, "\n")
    print("All hyperlinks are:", links, "\n")
    print("All probable descriptions are:", descriptions, "\n")
    print("Unclassified are:", rest)




#copied from stackoverflow: checks if string is in fact an int
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

# run classification
classify_val(all_triples)

Total found; dates  19 , images  22 , lone numbers  31 , hyperlinks  4 , descriptions  86  and unclassified  111 
 

All dates are : ['19 maart 1905', '1 september 1981', '6 april 1890', '23 december 1939', '7 september 1889', '31 december 1953', '23 juni 1912', '7 juni 1954', '150-250', '19 juli 1945', '1986-1994', '1994-1998', '1998-2002', '1998-2002', '2002-2006', '2006-2009', '2009-2011', '4 maart 1983', '10 december 1815'] 

All images are: ['Albert Speer Neurenberg.JPG', 'Andre Agassi 2005 US Clay Court.jpg', 'Groen_logo.png', 'Anthony Fokker 1912.jpg', 'File:Alan Turing photo.jpg', 'Alan Turing.jpg', 'Alpenrelief 01.jpg', 'Mont Blanc and Dome du Gouter.jpg', 'Alpen.jpg', 'AbelTasman.jpg', 'PvdA - Ad Melkert2002.jpg', 'Amsterdam Amstel 20041105.jpg', 'Amstelveen Oostermeer 2005 07 12.JPG', "Busto di Aristotele conservato a Palazzo Altaemps, Roma. Foto di Giovanni Dall'Orto.jpg", 'Atlantic Ocean.png', 'Atlantic_Ocean_surface.jpg', 'Earth Eastern Hemisphere.jpg', 'Location Andorra 

In [148]:
# Eerst alle attributen printen en kijken of veel dezelfde dingen zijn
def print_all(rdflist):
    for rdf in rdflist:
        print(rdf.p)
        
# Er valt te zien dat gbdat, geboortedatum etc hetzelfde zijn

def sort_attr(rdflist):
    all_attr = []
    for rdf in rdflist:
        all_attr.append(rdf.p)
    # alleen aparte lijst creeeren met juist genormeerde namen
    for attr in all_attr:
        index = all_attr.index(attr)
        # zoek elem met sterf erin en maak algemeen
        if 'sterf' in attr:
            #sterfdatum
            if 'dat' in attr:
                all_attr[index] = 'sterfdatum'
            #sterfplaats
            elif 'plaats' in attr:
                all_attr[index] = 'sterfplaats'
        #zoek elem met geboorte
        elif 'gb' in attr or 'geboorte' in attr:
              #gbdatum
            if 'dat' in attr:
                all_attr[index] = 'geboortedatum'
            #gbplaats
            elif 'plaats' in attr:
                all_attr[index] = 'geboorteplaats'
        #noem alle foto's afbeelding
        elif attr == 'foto':
            all_attr[index] = 'afbeelding'
    print(all_attr)

# Dit kan voor veel meer woorden gebeuren, 
# alleen moet wel handmatig worden gekeken hoe de termen van elkaar verschillen.
sort_attr(all_triples)

['afbeelding', 'onderschrift', 'geboortedatum', 'sterfdatum', 'geslacht', 'geboorteplaats', 'sterfplaats', 'zijde', 'org', 'spfunc', 'ATP-id', 'afbeelding', 'bijnaam', 'afbeelding', 'partijvoorzitter', 'vice-partijvoorzitter', 'functienaam1', 'functie1', 'functienaam2', 'functie2', 'functienaam3', 'functie3', 'functienaam4', 'functie4', 'functienaam5', 'functie5', 'zetelnaam1', 'zetels1', 'zetelstotaal1', 'verkiezingen1', 'zetelnaam2', 'zetels2', 'zetelstotaal2', 'verkiezingen2', 'zetelnaam3', 'zetels3', 'zetelstotaal3', 'verkiezingen3', 'zetelnaam4', 'zetels4', 'zetelstotaal4', 'verkiezingen4', 'zetelnaam5', 'zetels5', 'zetelstotaal5', 'verkiezingen5', 'zetelnaam6', 'zetels6', 'zetelstotaal6', 'verkiezingen6', 'zetelnaam7', 'zetels7', 'zetelstotaal7', 'verkiezingen7', 'exnaam', 'afbeelding', 'onderschrift', 'volledigenaam', 'geboortedatum', 'geboorteplaats', 'sterfdatum', 'sterfplaats', 'nationaliteit', 'portaal', 'onderschrift', 'volledigenaam', 'geboortedatum', 'geboorteplaats', 'st

# Reflectie

#### Huidig
Bij het parsen van de Infoboxen en het opzetten van de datastructuur is gebleken dat er veel ruis in de infoboxen zit, dit in de vorm van html tags, "lege" attributen enz. Dit zorgt voor een minder precieze analyse en ook voor dat de data onoverzichtelijk wordt voor mensen.
De opbouw van infoboxen is niet consistent (het gebruik van engelse en nederlandse termen door elkaar) zorgt voor een grote maat van onzuiverheid bij metingen
In het plot van hoe vaak de voorkomens zijn wordt bij veel dingen maar een voorkomen gemeten. Hierdoor vallen veel van de termen buiten het scherm. Dit wordt opgelost door alle termen die een keer voorkomen niet meer te laten zien. Dit is echter geen verantwoorde manier om de analyse toe te passen. 
Het terugkerende probleem blijft toch de ruis in de data die uit de infoboxes is geextraheerd.

Het inlezen van de grote file ging ook lastig. Alle hier gebouwde functies zijn dan ook alleen getest op de kleine file van Maarten Marx website.

----------------------------------------
#### Algemeen

Het lastigste onderdeel van het gehele proces vonden wij het omgaan met de gigantische datasets. Hoe lees je deze op de juiste manier in? Dit is bij ons dan ook amper gelukt. Maar door de opdrachten eerst met kleine sets te proberen kwamen we er toch nog redelijk uit.

Een ander lastig punt is dat niet alle informatie uniform is. Elke website en elke wiki-pagina doet het net op zijn eigen manier. Het is moeilijk om voor elk specifiek geval een speciale clausule te schrijven, en daarnaast is dat ook zeer tijdrovend. Het uitzoeken wat anders is alleen al kost zeer veel tijd. We hebben daarom maar een deel van het resultaat uniform gemaakt. Om te laten zien dat we het wel begrijpen, maar er geen dagen aan kwijt zijn. Ook, bij het verschil in de opmaak van websites, hebben we gewerkt met 'inspect element' van chrome om te zien waar de belangrijke info op de site stond.

Het kiezen van een datastructuur ging ons iets beter af, we hebben simpelweg onze eigen classes geimplementeerd. Maar we kunnen begrijpen dat anderen hier moeite mee hebben gehad.

